# Task 2
I will include some of the steps in the task 2 to start from the proper dataset

In [25]:
!pip install pandas
!pip install numpy

In [31]:
#import statements
import pandas as pd
import tensorflow as tf
import re
import spacy
nlp = spacy.load('en_core_web_sm')
from nltk.corpus import stopwords
import numpy as np
from functions import *

In [32]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [33]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 11.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [34]:
data = pd.read_csv('Data/Data_PCLab3_Twitter_Stock_Sentiment.csv')

In [35]:
data = word_count_col(data,'Text','pre_word_count')

In [36]:
documents = data["Text"].tolist()

In [37]:
nltk.download('stopwords')
%time clean_des = [preprocess(string) for string in documents[:]]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/valedipalo/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


CPU times: user 21.2 s, sys: 436 ms, total: 21.6 s
Wall time: 22 s


In [38]:
data['clean_text'] = clean_des
data = word_count_col(data,'clean_text','post_word_count')

In [39]:
data


,Text,Sentiment,pre_word_count,clean_text,post_word_count
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1,20,kicker watchlist xide tit soq pnk cpw bpz aj t...,15
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1,16,aap movie return fea geed indicator trade year...,9
2,user I'd be afraid to short AMZN - they are lo...,1,18,afraid short amzn look like near monopoly eboo...,10
3,MNTA Over 12.00,1,3,mnta over,2
4,OI Over 21.37,1,3,oi over,2
...,...,...,...,...,...
5786,Industry body CII said #discoms are likely to ...,0,19,industry body cii say discom likely suffer net...,13
5787,"#Gold prices slip below Rs 46,000 as #investor...",0,15,gold price slip below rs investor book profit ...,13
5788,Workers at Bajaj Auto have agreed to a 10% wag...,1,24,worker bajaj auto agree wage cut period april ...,11
5789,"#Sharemarket LIVE: Sensex off day’s high, up 6...",1,18,sharemarket live sensex day high point nifty t...,13


# Task 3


In [42]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, RNN, SimpleRNN
from sklearn.model_selection import train_test_split 


In [43]:
clean_text = data.clean_text.to_list()

In [44]:
#this cell is not useful for tokenizing and padding, but just to evaluate how many tokens we have before to proceed
#takes time to run; the result is 49483

#tokens = [[token.text for token in nlp(sentence)] for sentence in clean_text]
#count = 0
#for l in tokens:
#  for w in l:
#    count += 1 
#print(count)

In [45]:
# TOKENIZING: First trial, get a strange result as i am not able to include more words reducing the num_words which should represent the minimum frequency for a word to be included 
# tokenizer = Tokenizer(num_words=100000000, lower= 1, oov_token="<OOV>")
# tokenizer.fit_on_texts(clean_text)
# word_index = tokenizer.word_index
# print(len(word_index))

In [46]:
# TOKENIZING 
tk = Tokenizer()
tk.fit_on_texts(clean_text)

tk.num_words = 1000
word_index = tk.word_index
#print(word_index)

In [47]:
# SEQUENCES 
sequences = tk.texts_to_sequences(clean_text)
print(clean_text)
print(word_index)
print(sequences)

['kicker watchlist xide tit soq pnk cpw bpz aj trade method method see prev post', 'aap movie return fea geed indicator trade year awesome', 'afraid short amzn look like near monopoly ebook infrastructure service', 'mnta over', 'oi over', 'pgnx over', 'aap current downtrend break otherwise short term correction me term downtrend', 'monday relative weakness nyx win tie tap ice int bmc aon c chk biib', 'goog ower trend line channel test volume support', 'aap watch tomorrow ong entry', 'assume fcx open tomorrow above trigger buy still much like setup', 'really worry everyone expect market rally usually exact opposite happen every time shall see soon bac spx jpm', 'aap gamco arry haverty apple extremely cheap great video', 'maykiljil post agree msft go high possibly north', 'momentum come back etfc break resistance solid volume friday ong set', 'ha hit mean resume target level', 'gameplan shot today like trend break may c h break oc weekly trend break back july', 'fcx gapping well above id

In [48]:
# PADDING 
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding="post")
print(padded_inputs)
X = padded_inputs

[[302  33  25 ...   0   0   0]
 [  1 391 525 ...   0   0   0]
 [  2  69  10 ...   0   0   0]
 ...
 [710 632 186 ...   0   0   0]
 [ 45   4   3 ...   0   0   0]
 [ 45  49 640 ...   0   0   0]]


### LSTM 

In [49]:
model_LSTM = Sequential()
model_LSTM.add(Embedding(500, 120, input_length = X.shape[1]))
model_LSTM.add(SpatialDropout1D(0.4))
model_LSTM.add(LSTM(176, dropout=0.2, recurrent_dropout=0.2))
model_LSTM.add(Dense(2,activation='softmax'))
model_LSTM.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model_LSTM.summary())

Metal device set to: Apple M1


2022-10-02 19:57:55.661554: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-02 19:57:55.665421: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 21, 120)           60000     
                                                                 
 spatial_dropout1d (SpatialD  (None, 21, 120)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 176)               209088    
                                                                 
 dense (Dense)               (None, 2)                 354       
                                                                 
Total params: 269,442
Trainable params: 269,442
Non-trainable params: 0
_________________________________________________________________
None


In [50]:
y = pd.get_dummies(data['Sentiment'])

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, random_state = 42)

In [53]:
batch_size=32
# model_LSTM.fit(X_train, y_train, epochs = 10, batch_size=batch_size, verbose = 'auto')
model_LSTM.fit(X_train, y_train, epochs = 1, batch_size=batch_size, verbose = 'auto')

163/163 [==============================] - 201s 1s/step - loss: 0.4924 - accuracy: 0.7651


In [54]:
loss, accuracy = model_LSTM.evaluate(X_test,y_test)
print(accuracy)

2022-10-02 21:50:28.349087: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 2s 69ms/step - loss: 0.4956 - accuracy: 0.7672
0.767241358757019


In [57]:
# Confusion Matrix 
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
import matplotlib.pyplot as plt

In [58]:
y_test = y_test.iloc[:,1]
y_pred = model_LSTM.predict(X_test)
confusion_matrix(y_test, y_pred)

IndexingError: Too many indexers

### Bi-LSTM 

In [59]:
from keras.layers import Bidirectional

In [60]:
model_BI = Sequential()
model_BI.add(Embedding(500, 120, input_length = X.shape[1]))
model_BI.add(SpatialDropout1D(0.4))
model_BI.add(Bidirectional(LSTM(176, dropout=0.2, recurrent_dropout=0.2)))
model_BI.add(Dense(2,activation='softmax'))
model_BI.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model_BI.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 21, 120)           60000     
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 21, 120)          0         
 lDropout1D)                                                     
                                                                 
 bidirectional (Bidirectiona  (None, 352)              418176    
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 2)                 706       
                                                                 
Total params: 478,882
Trainable params: 478,882
Non-trainable params: 0
_________________________________________________________________
None


In [62]:
batch_size=32
# model_BI.fit(X_train, y_train, epochs = 10, batch_size=batch_size, verbose = 'auto')
model_BI.fit(X_train, y_train, epochs = 1, batch_size=batch_size, verbose = 'auto')

163/163 [==============================] - 360s 2s/step - loss: 0.5646 - accuracy: 0.7008


In [63]:
loss, accuracy = model_BI.evaluate(X_test,y_test)
print(accuracy)

ValueError: in user code:

    File "/Users/valedipalo/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1727, in test_function  *
        return step_function(self, iterator)
    File "/Users/valedipalo/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1713, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/valedipalo/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1701, in run_step  **
        outputs = model.test_step(data)
    File "/Users/valedipalo/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1667, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/valedipalo/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "/Users/valedipalo/minifor

In [64]:
y_pred = model_BI.predict(X_test)

2022-10-02 21:59:09.419798: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 2s 93ms/step


In [65]:
y_test = y_test.iloc[:,1]
y_test=np.argmax(y_test, axis=1)
confusion_matrix(y_test, y_pred)

IndexingError: Too many indexers



---



In [66]:
model_SimpleRNN = Sequential()
model_SimpleRNN.add(Embedding(500, 120, input_length = X.shape[1]))
model_SimpleRNN.add(SpatialDropout1D(0.4))
model_SimpleRNN.add(SimpleRNN(176))
model_SimpleRNN.add(Dense(2,activation='softmax'))
model_SimpleRNN.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model_SimpleRNN.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 21, 120)           60000     
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 21, 120)          0         
 lDropout1D)                                                     
                                                                 
 simple_rnn (SimpleRNN)      (None, 176)               52272     
                                                                 
 dense_2 (Dense)             (None, 2)                 354       
                                                                 
Total params: 112,626
Trainable params: 112,626
Non-trainable params: 0
_________________________________________________________________
None


In [67]:
batch_size=32
model_SimpleRNN.fit(X_train, y_train, epochs = 1, batch_size=batch_size, verbose = 'auto')

2022-10-02 21:59:50.494734: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


163/163 [==============================] - 38s 228ms/step - loss: 0.6131 - accuracy: 0.6663


In [ ]:
model_SimpleRNN.evaluate(X_test,y_test)

19/19 [==============================] - 0s 4ms/step - loss: 0.6706 - accuracy: 0.7431


[0.6705605983734131, 0.7431034445762634]

In [68]:
y_pred = model_SimpleRNN.predict(X_test)

 3/19 [===>..........................] - ETA: 0s

2022-10-02 22:01:00.185563: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 1s 31ms/step


In [69]:
y_pred=np.argmax(y_pred, axis=1)


In [70]:
loss, accuracy = model_SimpleRNN.evaluate(X_test,y_test)
print(accuracy)

ValueError: in user code:

    File "/Users/valedipalo/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1727, in test_function  *
        return step_function(self, iterator)
    File "/Users/valedipalo/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1713, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/valedipalo/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1701, in run_step  **
        outputs = model.test_step(data)
    File "/Users/valedipalo/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1667, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/valedipalo/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "/Users/valedipalo/minifor

In [71]:
y_test = y_test.iloc[:,1]
y_test=np.argmax(y_test, axis=1)
confusion_matrix(y_test, y_pred)

IndexingError: Too many indexers

array([[129,  83],
       [ 66, 302]])